# Semantic Search - Wikipedia Question-Answer-Retrieval

This examples demonstrates the setup for Question-Answer-Retrieval.

You can input a query or a question. The script then uses semantic search
to find relevant passages in Simple English Wikipedia (as it is smaller and fits better in RAM).

As model, we use: nq-distilbert-base-v1

It was trained on the Natural Questions dataset, a dataset with real questions from Google Search together with annotated data from Wikipedia providing the answer. For the passages, we encode the Wikipedia article tile together with the individual text passages.

In [ ]:
%%capture
!pip install -U sentence-transformers

In [ ]:
%%capture
!pip install datasets

In [ ]:
from datasets import load_dataset
ds = load_dataset("Coder-Dragon/wikipedia-movies", split='train[:1000]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import itertools

for movie in itertools.islice(ds, 3):
  title, plot = movie['Title'], movie['Plot']
  print(title, '--->' ,plot)

Kansas Saloon Smashers ---> A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]
Love by the Light of the Moon ---> The moon, painted with a smiling face hangs over a park at night. A young couple walking past a fence learn on a railing and look up. The moon smiles. They embrace, and the moon's smile gets bigger. They then sit down on a bench by a tree. The moon's view is blocked, causing him to frown. In the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everything better

##Bi-Encoder: Encoding Wikipedia movie plots for semantic search.

In [ ]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import time
import gzip
import os
import torch

ds = load_dataset("Coder-Dragon/wikipedia-movies", split='train[:1000]')

if not torch.cuda.is_available():
  print("Warning: No GPU found. Please add GPU to your notebook")


# We use the Bi-Encoder to encode all passages, so that we can use it with sematic search
model_name = 'nq-distilbert-base-v1'
bi_encoder = SentenceTransformer(model_name)
top_k = 5  # Number of passages we want to retrieve with the bi-encoder

passages = []
for movie in ds:
   passages.append([movie['Title'], movie['Plot']])

# If you like, you can also limit the number of passages you want to use
print("Passages:", len(passages))

corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/540 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Passages: 1000


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

##Semantic search: Find relevant passages using Bi-Encoder embeddings.

In [ ]:
def search(query):
    # Encode the query using the bi-encoder and find potentially relevant passages
    start_time = time.time()
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query
    end_time = time.time()

    # Output of top-k hits
    print("Input question:", query)
    print("Results (after {:.3f} seconds):".format(end_time - start_time))
    for hit in hits:
        print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']]))

##Bi-Encoder semantic search: Retrieving top-k relevant passages for query

In [ ]:
def search(query):
    # Encode the query using the bi-encoder and find potentially relevant passages
    start_time = time.time()
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query
    end_time = time.time()

    # Output of top-k hits
    print("Input question:", query)
    print("Results (after {:.3f} seconds):".format(end_time - start_time))
    for hit in hits:
        print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']]))

In [ ]:
# Ground Truth: Nanook of the North
search(query = "Documentaries showcasing indigenous peoples' survival and daily life in Arctic regions")

Input question: Documentaries showcasing indigenous peoples' survival and daily life in Arctic regions
Results (after 0.130 seconds):
	0.457	['Nanook of the North', 'The documentary follows the lives of an Inuk, Nanook, and his family as they travel, search for food, and trade in the Ungava Peninsula of northern Quebec, Canada. Nanook; his wife, Nyla; and their family are introduced as fearless heroes who endure rigors no other race could survive. The audience sees Nanook, often with his family, hunt a walrus, build an igloo, go about his day, and perform other tasks.']
	0.264	['David Copperfield', '"David Copperfield consists of three reels and as three separate films, released in three consecutive weeks, with three different titles: The Early Life of David Copperfield, Little Em\'ly and David Copperfield and The Loves of David Copperfield.[4]']
	0.251	['Straight Shooting', 'At the end of the 19th century in the Far West, a farmer is fighting for his right to plough the plains. In ord

In [ ]:
# Ground Truth: The Lucky Horseshoe
search(query = "Western romance")

Input question: Western romance
Results (after 0.016 seconds):
	0.305	['The Great Gatsby', "An adaptation of F. Scott Fitzgerald's Long Island-set novel, where Midwesterner Nick Carraway is lured into the lavish world of his neighbor, Jay Gatsby. Soon enough, however, Carraway will see through the cracks of Gatsby's nouveau riche existence, where obsession, madness, and tragedy await."]
	0.295	["Youth's Endearing Charm", 'The film is about a court case and embezzlement.']
	0.283	['Frankenstein', 'Described as "a liberal adaptation of Mrs. Shelley\'s famous story", the plot description in the Edison Kinetogram was:[3]']
	0.280	['High Society Blues', 'A new country family comes to live among established wealthy neighbors.']
	0.277	['Romance', 'As described in a film publication,[2] a youth (Arthur Rankin) in the prologue seeks advice from his grandfather (Sydney), who then recalls a romance of his own youth which is then shown as a flashback. A priest (Sydney) is in love with an Italian 

In [ ]:
# Ground Truth: Sahara
search(query = "Silent film about a Parisian star moving to Egypt, leaving her husband for a baron, and later reconciling after finding her family in poverty in Cairo.")

Input question: Silent film about a Parisian star moving to Egypt, leaving her husband for a baron, and later reconciling after finding her family in poverty in Cairo.
Results (after 0.012 seconds):
	0.449	['Married in Hollywood', "A showgirl, part of a troupe, tours Europe where she falls in love with a Balkan prince. The prince's parents disapprove and attempt to put a stop to the romance. A revolution occurs and the prince and the showgirl elope to Hollywood."]
	0.394	['The King on Main Street', 'King Serge IV of Molvania (Menjou) comes to a small American town, and falls in love with one of its residents, Mary Young (Love).[3][4]']
	0.392	['Sahara', "Silent film femme fatale, Louise Glaum, portrays the role of Mignon, a Parisian music hall celebrity. Mignon marries a young American civil engineer, John Stanley, portrayed by Matt Moore. Stanley is transferred to Egypt to work on an engineering project in the Sahara. Mignon and her son, portrayed by Pat Moore, join Stanley in the des

In [ ]:
# Ground Truth: Ask Father
search(query = "Comedy film, office disguises, boss's daughter, elopement.")

Input question: Comedy film, office disguises, boss's daughter, elopement.
Results (after 0.013 seconds):
	0.403	["Youth's Endearing Charm", 'The film is about a court case and embezzlement.']
	0.400	['Dressed to Kill', 'The gang of a mob boss grow suspicious of his new girlfriend. She\'s a beautiful young girl and they don\'t believe she would actually associate with the mob and wonder if she\'s really a police "plant". The mobsters dress nattily to not appear "out of place" in the ritzy neighborhoods prior to a heist.']
	0.398	['The Boy Friend', 'Comedy about a small-town girl unhappy with her family, and a boy trying to please her by throwing a big party.']
	0.390	['A Little Journey', 'A girl travelling by train to meet her boyfriend meets another young man and falls in love with him.']
	0.373	['This Is Heaven', "Vilma Banky portrays a newly arrived Hungarian immigrant who learns to accustom herself to the new and strange life she finds in New York. The story gave Miss Banky moments

In [ ]:
# Ground Truth:  Cleopatra
search(query = "Lost film, Cleopatra charms Caesar, plots world rule, treasures from mummy, revels with Antony, tragic end with serpent in Alexandria.")

Input question: Lost film, Cleopatra charms Caesar, plots world rule, treasures from mummy, revels with Antony, tragic end with serpent in Alexandria.
Results (after 0.013 seconds):
	0.537	['Cleopatra', "Because the film has been lost, the following summary is reconstructed from a description in a contemporary film magazine.\r\nCleopatra (Bara), the Siren of Egypt, by a clever ruse reaches Caesar (Leiber) and he falls victim to her charms. They plan to rule the world together, but then Caesar falls. Cleopatra's life is desired by the church, as the wanton woman's rule has become intolerable. Pharon (Roscoe), a high priest, is given a sacred dagger to take her life. He gives her his love instead and, when she is in need of some money, leads her to the tomb of his ancestors, where she tears the treasure from the breast of the mummy. With this wealth she goes to Rome to meet Antony (Hall). He leaves the affairs of state and travels to Alexandria with her, where they revel. Antony is recal

In [ ]:
# Ground Truth: Captain Alvarez
search(query = "Denis Gage Deane-Tanner")

Input question: Denis Gage Deane-Tanner
Results (after 0.011 seconds):
	0.365	['Souls for Sale', 'Remember "Mem" Steddon (Eleanor Boardman) marries Owen Scudder (Lew Cody) after a whirlwind courtship. However, on their wedding night, she has a change of heart. When the train taking them to Los Angeles stops for water, she impulsively and secretly gets off in the middle of the desert. Strangely, when Scudder realizes she is gone, he does not have the train stopped.\r\nMem sets off in search of civilization. Severely dehydrated, she sees an unusual sight: an Arab on a camel. It turns out to be actor Tom Holby (Frank Mayo); she has stumbled upon a film being shot on location. When she recuperates, she is given a role as an extra. Both Holby and director Frank Claymore (Richard Dix) are attracted to her. However, when filming ends, she does not follow the troupe back to Hollywood, but rather gets a job at a desert inn.\r\nMeanwhile, Scudder is recognized and arrested at the train station. 

##Evaluation Metrics

In [ ]:
# Recall@1

(1 + 0 + 0 + 0 + 1 + 0)/6

0.3333333333333333

In [ ]:
# MRR

(1 + 0 + (1/3) + 0 + 1 + 0)/6

0.38888888888888884